<a href="https://colab.research.google.com/github/narendra974/AIMLOPS_IISC/blob/main/NMT_TRANSFORMERS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/DATASETS/ENG_DEU_DATA/

/content/drive/MyDrive/DATASETS/ENG_DEU_DATA


In [4]:
!pip install pycld2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycld2: filename=pycld2-0.41-cp310-cp310-linux_x86_64.whl size=9904028 sha256=aa28f740ab282cbc483a2a7b63ed59c6c4401162daf3289b79b7f1b6df91e9ea
  Stored in directory: /root/.cache/pip/wheels/be/81/31/240c89c845e008a93d98542325270007de595bfd356eb0b06c
Successfully built pycld2


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import pycld2 as cld2
import spacy
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import regex as re

import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [6]:
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
!python -m spacy download de_core_news_sm
gernlp = spacy.load('de_core_news_sm')
engnlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


2023-12-24 12:10:26.848511: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 33.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [7]:
def removestop(text,stopwords):
  raw = text.split()
  words = [word for word in raw if not word in stopwords]
  cleanwords = ' '.join(words)
  return cleanwords

def tolower(text):
  return text.lower()

def removespecial(text):
  te1 = re.sub("\s+"," ",text)
  te2 = re.sub('\n', '', te1)
  te3 = re.sub('\r', '', te2)
  te4 = re.sub("[0-9]","",te3)
  te5 = re.sub("()@%^&*-_,/\{}[?|$|.|!]","",te4)
  te6 = re.sub(r"[\p{Cc}\p{Cs}]+","",te5)
  te7 = re.sub(r'[^\w\s]','', te6)
  te8 = re.sub("[^a-zA-Z ]","",te7)
  return te7

def removeurl(text):
  return re.sub('https?://\S+|www\.\S+', '', text)

def clean_text(text):
    text = text.lower()
    pattern = re.compile(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?")
    text = re.sub(pattern,' ',text).strip()
    return text

In [8]:
basefolder =  "/content/drive/MyDrive/DATASETS/ENG_DEU_DATA/"
# germanfiles = ["commoncrawl_de_en.txt","europarl-v7_de_en.txt","news-commentary-v9_de_en.txt"]
# engfiles = ["commoncrawl_en_de.txt","europarl-v7_en_de.txt","news-commentary-v9_en_de.txt"]
germanfiles = ["news-commentary-v9_de_en.txt"]
engfiles = ["news-commentary-v9_en_de.txt"]

In [9]:
def read_files(fileloc, language):
  with open(fileloc,"rb") as f:
    f_lines = f.readlines()
  df = pd.DataFrame(f_lines)
  dfc = df.set_axis([language],axis=1)
  dfc[language] = dfc[language].str.decode("utf-8")
  return dfc

In [10]:
dfappend=pd.DataFrame()
for efile in range(len(germanfiles)):
  germanfilepath = basefolder+germanfiles[efile]
  print(germanfilepath)
  germandff = read_files(germanfilepath,"german")
  engfilepath = basefolder+engfiles[efile]
  engdff = read_files(engfilepath,"english")
  print(germandff.shape)
  print(engdff.shape)
  dfconcat = pd.concat([germandff, engdff],axis="columns")
  dfappend=pd.concat([dfappend, dfconcat])

/content/drive/MyDrive/DATASETS/ENG_DEU_DATA/news-commentary-v9_de_en.txt
(201288, 1)
(201288, 1)


In [11]:
dfappend.isna().sum()

german     0
english    0
dtype: int64

In [12]:
dfappend.duplicated().sum()

430

In [13]:
dfappend.drop_duplicates(subset=None, keep='first', inplace=True)
dfappend.shape

(200858, 2)

In [14]:
german_stop_words = stopwords.words('german')
english_stop_words = stopwords.words('english')

dfappend['english_clean'] = dfappend['english'].apply(lambda x: removestop(x,english_stop_words))
dfappend['german_clean'] = dfappend['german'].apply(lambda x: removestop(x,german_stop_words))
dfappend['german_clean'] = dfappend['german_clean'].apply(lambda x: removespecial(x))
dfappend['english_clean'] = dfappend['english_clean'].apply(lambda x: removespecial(x))

In [15]:
def langdet(x):
  isReliable, textBytesFound, details = cld2.detect(x)
  return(details[0][1])

In [16]:
dfappend['is_eng'] = dfappend['english_clean'].apply(lambda x: langdet(x))
dfappend['is_ger'] = dfappend['german_clean'].apply(lambda x: langdet(x))

In [17]:
display(dfappend["is_ger"].value_counts())

de     195236
un       4267
en        861
nl         83
lb         83
ru         83
nn         37
da         35
no         16
na         13
fy         12
af         11
la          7
sv          7
id          7
pt          6
ms          6
fr          5
tk          5
war         5
ie          5
ro          5
rm          5
sk          4
fi          4
tr          3
it          3
pl          3
eu          3
es          3
vo          3
gv          2
cs          2
et          2
tn          2
ha          2
lt          2
gn          2
hr          2
rw          2
ia          1
mfe         1
mt          1
bi          1
tl          1
lv          1
zzp         1
ca          1
az          1
sco         1
fo          1
jw          1
ts          1
aa          1
Name: is_ger, dtype: int64

In [18]:
display(dfappend["is_eng"].value_counts())

en     196526
un       4140
sco        26
ru         24
da         20
zh         17
ca         10
ms          7
la          7
id          5
tr          5
oc          5
rw          5
rm          4
gn          3
pt          3
gv          3
zzp         3
es          3
de          2
co          2
fr          2
pl          2
ro          2
bs          2
mfe         2
nn          2
uz          2
ie          2
ha          2
hmn         1
az          1
lt          1
mg          1
af          1
vo          1
gl          1
no          1
fj          1
sq          1
sn          1
ts          1
sr          1
ceb         1
kha         1
hu          1
ga          1
bi          1
war         1
tk          1
Name: is_eng, dtype: int64

In [19]:
dfappendclean = dfappend[(dfappend.is_eng == 'en') & (dfappend.is_ger =='de')]

In [20]:
dfappendclean.shape

(192441, 6)

In [21]:
display(dfappendclean)

,german,english,english_clean,german_clean,is_eng,is_ger
1,"SAN FRANCISCO – Es war noch nie leicht, ein ra...",SAN FRANCISCO – It has never been easy to have...,SAN FRANCISCO It never easy rational conversa...,SAN FRANCISCO Es nie leicht rationales Gesprä...,en,de
2,In letzter Zeit allerdings ist dies schwierige...,"Lately, with gold prices up more than 300% ove...",Lately gold prices last decade harder ever,In letzter Zeit allerdings schwieriger je Gold...,en,de
3,Erst letzten Dezember verfassten meine Kollege...,"Just last December, fellow economists Martin F...",Just last December fellow economists Martin Fe...,Erst letzten Dezember verfassten Kollegen Mart...,en,de
4,"Und es kam, wie es kommen musste.\n",Wouldn’t you know it?\n,Wouldnt know it,Und kam kommen musste,en,de
5,Seit der Veröffentlichung ihrer Artikel ist de...,"Since their articles appeared, the price of go...",Since articles appeared price gold moved still...,Seit Veröffentlichung Artikel Goldpreis gestiegen,en,de
...,...,...,...,...,...,...
201283,Das bleibt eine der größten Errungenschaften i...,Their achievement remains one of the greatest ...,Their achievement remains one greatest recent ...,Das bleibt größten Errungenschaften jüngeren G...,en,de
201284,Gleichzeitig scheint sich Zumas revolutionäre ...,"At the same time, Zuma’s revolutionary generat...",At time Zumas revolutionary generation still s...,Gleichzeitig scheint Zumas revolutionäre Gener...,en,de
201285,"In einer Region, wo die älteren Menschen sehr ...","In a region that reveres the elderly, Zuma’s a...",In region reveres elderly Zumas attachment rur...,In Region älteren Menschen verehrt werden Zuma...,en,de
201286,Drei von zehn Südafrikanern sind jünger als 15...,Three in ten South Africans are younger than 1...,Three ten South Africans younger meaning live...,Drei zehn Südafrikanern jünger bedeutet Tag A...,en,de


In [22]:
dfappendclean["engcount"]=dfappendclean['english_clean'].str.split().str.len()

<ipython-input-22-b3f513c107bd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfappendclean["engcount"]=dfappendclean['english_clean'].str.split().str.len()


In [23]:
train_n=10000
total = int(1.20 * train_n)
dfappendclean = dfappendclean.sample(total)
dfappendclean_val = dfappendclean[train_n:total]
dfappendclean = dfappendclean[:train_n]

In [24]:
bos_string = 'bos '
eos_string = ' eos'
dfappendclean['english_clean'] = bos_string + dfappendclean['english_clean'].astype(str) + eos_string
dfappendclean['german_clean'] = bos_string + dfappendclean['german_clean'].astype(str) + eos_string

en_list =  dfappendclean['english_clean'].astype(str).tolist()
ge_list =  dfappendclean['german_clean'].astype(str).tolist()
en_list_val =  dfappendclean_val['english_clean'].astype(str).tolist()
ge_list_val =  dfappendclean_val['german_clean'].astype(str).tolist()


In [25]:
display(en_list[0:5])

['bos But whatever level nominal interest rates monetarypolicy stance established way often poorly transmitted economy particularly times acute crisis eos',
 'bos That makes frightening ordinary criminals eos',
 'bos The international communitys old approach prioritize stability democracy pursue IsraeliArab peace completely separate diplomatic track eos',
 'bos This could story Romanian revolution  years ago eos',
 'bos In Europe crisis management mechanisms must put place manage deeply integrated nature  Europes banking assets held crossborder banking groups eos']

In [26]:
display(ge_list[0:5])

['bos Aber unabhängig Nominalverzinsung festgelegte geldpolitische Kurs oft geringem Maße Wirtschaft übertragen besonders Zeiten akuter Krisen eos',
 'bos Das macht Terroristen Furcht erregender gewöhnliche Kriminelle eos',
 'bos Der alte Ansatz internationalen Gemeinschaft bestand darin Stabilität Demokratie stellen israelischarabischen Frieden völlig diplomatischen Schiene verfolgen eos',
 'bos Dies Geschichte rumänischen Revolution  Jahren sein eos',
 'bos Es müssen Europa Mechanismen Krisenmanagement eingerichtet werden hochintegrierten Charakter europäischen Kapitalmärkte Rechnung tragen  Gesamtbilanzsumme Europa tätigen Kreditinstitute grenzübergreifenden Bankkonzernen gehalten eos']

In [27]:
MAX_TOKENS=128
en_tokenizer = Tokenizer();
en_tokenizer.fit_on_texts(dfappendclean["english_clean"])

en_bos_index = en_tokenizer.word_index['bos']
print(en_bos_index)
en_eos_index = en_tokenizer.word_index['eos']
print(en_eos_index)

1
2


In [28]:
ge_tokenizer = Tokenizer();
ge_tokenizer.fit_on_texts(dfappendclean["german_clean"])

ge_bos_index = ge_tokenizer.word_index['bos']
print(ge_bos_index)
ge_eos_index = ge_tokenizer.word_index['eos']
print(ge_eos_index)

1
2


In [29]:
MAX_TOKENS=128
en = en_tokenizer.texts_to_sequences(en_list)     # Output is ragged.
en = tf.keras.utils.pad_sequences(en, maxlen=MAX_TOKENS, padding='post')

ge = ge_tokenizer.texts_to_sequences(ge_list)
ge = tf.keras.utils.pad_sequences(ge, maxlen=MAX_TOKENS+1, padding='post')

en_val = en_tokenizer.texts_to_sequences(en_list_val)     # Output is ragged.
en_val = tf.keras.utils.pad_sequences(en_val, maxlen=MAX_TOKENS, padding='post')

ge_val = ge_tokenizer.texts_to_sequences(ge_list_val)
ge_val = tf.keras.utils.pad_sequences(ge_val, maxlen=MAX_TOKENS+1, padding='post')


In [30]:
dataset = tf.data.Dataset.from_tensor_slices((en, ge))
dataset_val = tf.data.Dataset.from_tensor_slices((en_val, ge_val))

In [31]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64
def prepare_batch(en, ge):

    ge_inputs = ge[:, :-1]
    ge_labels = ge[:, 1:]

    return (en, ge_inputs), ge_labels

def make_batches(ds):
  return (
      ds
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(prepare_batch, tf.data.AUTOTUNE)
      .prefetch(buffer_size=tf.data.AUTOTUNE))

# Create training and validation set batches.
dataset_batches = make_batches(dataset)
dataset_batches_val = make_batches(dataset_val)

In [32]:
for (pt, en), en_labels in dataset_batches.take(1):
  break

print(pt.shape)
print(en.shape)
print(en_labels.shape)

(64, 128)
(64, 128)
(64, 128)


In [33]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1)

  return tf.cast(pos_encoding, dtype=tf.float32)

In [34]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
    self.pos_encoding = positional_encoding(length=128, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x


In [35]:
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

In [36]:
class CrossAttention(BaseAttention):
  def call(self, x, context):
    attn_output, attn_scores = self.mha(
        query=x,
        key=context,
        value=context,
        return_attention_scores=True)

    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [37]:
class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [38]:
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [39]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x)
    return x


In [40]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

In [41]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads,
               dff, vocab_size, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(
        vocab_size=vocab_size, d_model=d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.

    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.

In [42]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

In [43]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x

In [44]:
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size,
                           dropout_rate=dropout_rate)

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    context, x  = inputs
    context = self.encoder(context)  # (batch_size, context_len, d_model)

    x = self.decoder(x, context)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits

In [45]:
num_layers = 3
d_model = 128
dff = 128
num_heads = 8
dropout_rate = 0.1

In [46]:
en_vocab_size = len(en_tokenizer.word_index)+1;
ge_vocab_size = len(ge_tokenizer.word_index)+1;

transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size = en_vocab_size,
    target_vocab_size = ge_vocab_size,
    dropout_rate=dropout_rate)

In [47]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [48]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [49]:
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [50]:
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [51]:
transformer.fit(dataset_batches, epochs=30, validation_data=dataset_batches_val)

Epoch 1/30
157/157 [==============================] - 691s 4s/step - loss: 10.1454 - masked_accuracy: 0.0482 - val_loss: 9.9714 - val_masked_accuracy: 0.0000e+00
Epoch 2/30
157/157 [==============================] - 672s 4s/step - loss: 9.5872 - masked_accuracy: 0.0690 - val_loss: 9.2828 - val_masked_accuracy: 0.0000e+00
Epoch 3/30
157/157 [==============================] - 654s 4s/step - loss: 8.8872 - masked_accuracy: 0.0691 - val_loss: 8.8726 - val_masked_accuracy: 0.0000e+00
Epoch 4/30
157/157 [==============================] - 651s 4s/step - loss: 8.5449 - masked_accuracy: 0.0701 - val_loss: 8.8495 - val_masked_accuracy: 1.3140e-04
Epoch 5/30
157/157 [==============================] - 651s 4s/step - loss: 8.3398 - masked_accuracy: 0.0785 - val_loss: 8.7831 - val_masked_accuracy: 0.0013
Epoch 6/30
157/157 [==============================] - 654s 4s/step - loss: 8.1418 - masked_accuracy: 0.0804 - val_loss: 8.7936 - val_masked_accuracy: 0.0050
Epoch 7/30
157/157 [=====================

In [57]:
transformer.summary()

Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  3965056   
                                                                 
 decoder (Decoder)           multiple                  6934784   
                                                                 
 dense_12 (Dense)            multiple                  3697140   
                                                                 
Total params: 14,596,980
Trainable params: 14,596,980
Non-trainable params: 0
_________________________________________________________________


In [52]:
class Translator(tf.Module):
  def __init__(self, en_tokenizer, ge_tokenizer, transformer):
    self.en_tokenizer = en_tokenizer
    self.ge_tokenizer = ge_tokenizer
    self.transformer = transformer

  def __call__(self, sentence, max_length=MAX_TOKENS):

    # assert isinstance(sentence, tf.Tensor)
    # if len(sentence.shape) == 0:
    #  sentence = sentence[tf.newaxis]
    sentence = self.en_tokenizer.texts_to_sequences(sentence)
    sentence = tf.keras.utils.pad_sequences(sentence, maxlen=MAX_TOKENS, padding='post')
    encoder_input = sentence
    start = tf.constant(ge_bos_index, dtype=tf.int64)[tf.newaxis]
    end = ge_eos_index

    # `tf.TensorArray` is required here (instead of a Python list), so that the
    # dynamic-loop can be traced by `tf.function`.
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)

    for i in tf.range(max_length-1):
      output = tf.transpose(output_array.stack())
      predictions = self.transformer([encoder_input, output], training=False)

      # Select the last token from the `seq_len` dimension.
      predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

      predicted_id = tf.argmax(predictions, axis=-1)

      # Concatenate the `predicted_id` to the output which is given to the
      # decoder as its input.
      output_array = output_array.write(i+1, predicted_id[0])
      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    # The output shape is `(1, tokens)`.
    # print(output[0].numpy())
    text = ge_tokenizer.sequences_to_texts([output[0].numpy()])  # Shape: `()`.

    # `tf.function` prevents us from using the attention_weights that were
    # calculated on the last iteration of the loop.
    # So, recalculate them outside the loop.
    self.transformer([encoder_input, output[:,:-1]], training=False)
    attention_weights = self.transformer.decoder.last_attn_scores

    return text, attention_weights

In [53]:
translator = Translator(en_tokenizer, ge_tokenizer, transformer)

def print_translation(sentence, translated_text, ground_truth):
  print(f'{"Input":15s}: {sentence}')
  print(f'{"Prediction":15s}: {translated_text[0]}')
  print(f'{"Ground truth":15s}: {ground_truth}')

In [54]:
sentence = ['bos The Chinese symbol form carries reasoning due use ideograms eos']
ground_truth = 'bos Das chinesische Symbol Form verkörpert  aufgrund Verwendung Ideogrammen Chinesischen  Denkweise eos'
translated_text, attention_weights = translator(sentence)
print_translation(sentence, translated_text, ground_truth)

Input          : ['bos The Chinese symbol form carries reasoning due use ideograms eos']
Prediction     : bos das chinesen aufnehmen bevor zweites liberia schaden zuzufügen eos
Ground truth   : bos Das chinesische Symbol Form verkörpert  aufgrund Verwendung Ideogrammen Chinesischen  Denkweise eos


In [55]:
sentence = ['bos Energy carbon taxes produce less economic pain gain conventional taxes can eos']
ground_truth = 'bos Energie COSteuern wirtschaftlicher Hinsicht weniger schmerzhaft dabei einträglicher herkömmliche Steuern eos'
translated_text, attention_weights = translator(sentence)
print_translation(sentence, translated_text, ground_truth)

Input          : ['bos Energy carbon taxes produce less economic pain gain conventional taxes can eos']
Prediction     : bos die energiecharta bereits weniger stärke erwarten gewinnen teilzuhaben stammaktionären zufließen steuern erhöhen eos
Ground truth   : bos Energie COSteuern wirtschaftlicher Hinsicht weniger schmerzhaft dabei einträglicher herkömmliche Steuern eos


In [56]:
sentence = ['bos In India newborn health forms part national Reproductive Child Health Program eos']
ground_truth = 'bos In Indien Gesundheit Neugeborener Teil Programms Fortpflanzungsmedizin Kindergesundheit eos'
translated_text, attention_weights = translator(sentence)
print_translation(sentence, translated_text, ground_truth)

Input          : ['bos In India newborn health forms part national Reproductive Child Health Program eos']
Prediction     : bos in indien gesundheit bemerkt erst bilden beitritt krisenzeiten zugeführt werden eos
Ground truth   : bos In Indien Gesundheit Neugeborener Teil Programms Fortpflanzungsmedizin Kindergesundheit eos
